Ce script permet de scraper les fiches métiers de l'Onisep sur le site de l'Onisep.
Les liens vers les fiches viennent d'un document disponible sur data.gouv.fr : http://www.data.gouv.fr/fr/datasets/liste-des-metiers-onisep/
Le résultat donne des fiches au format JSON copiés dans le dossier 'Scraping output'.

In [4]:
import os
import json
import numpy as np
import math
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup

In [5]:
# Dossier de travail
dir_path = os.getcwd()
# Dossier data
data_dir_path = os.path.join(dir_path, '0 - data')
# Dossier scraping output
output_dir_path = os.path.join(dir_path, '00 - Scraping output')

In [6]:
# Liste des liens vers les métiers
metiers_file_path = os.path.join(data_dir_path, 'Onisep_metier_link.csv')
metiers_links = pd.read_csv(metiers_file_path, header=0, sep=";")
metiers_links.head()

,libellé métier,lien site onisep.fr
0,technicien/ne de fabrication de mobilier et de...,http://www.onisep.fr/http/redirection/metier/i...
1,agenceur/euse de cuisines et salles de bains,http://www.onisep.fr/http/redirection/metier/i...
2,chef/fe de fabrication des industries graphiques,http://www.onisep.fr/http/redirection/metier/i...
3,conducteur/trice de machines à imprimer,http://www.onisep.fr/http/redirection/metier/i...
4,opérateur/trice prépresse,http://www.onisep.fr/http/redirection/metier/i...


In [7]:
# Cette fonction scrap une fiche métier de l'Onisep et permet de récupérer un dictionnaire Python avec
#  toutes les informations disponibles
def parse_job_page(link):
    #print('GET '+ link)
    
    try:
        page = request.urlopen(link).read().decode('utf-8')

        page = page.replace('€', 'euros')  # because encode/decode error
        page = page.replace(u"\u2019", "'")  # because encode/decode error
        page = page.replace(u"\u2026", "é")  # because encode/decode error
        page = page.replace(u"\u0153", "oe")  # because encode/decode error
        page = page.replace(u"\u00a0", "À")  # because encode/decode error

        page = page.replace('\n', '')  # remove new line
        page = page.replace('  ', '')  # remove big spaces (to be more human readable)
        #file = os.path.join(fiches_dir_path, name.replace('/', '-').replace(' ', '_') + '.html')
        #with open(file, 'w', encoding='utf8') as f:
        #    f.write(page)

        soup = BeautifulSoup(page, 'html.parser')
   
        metier = {}
        metier["Métier"] = soup.h1.extract().get_text()
        
        # On récupère la "fiche" avec Salaire débutant, centre(s) d'intérêt, etc
        fiche = soup.find(name = 'ul', attrs={'class':'oni_last'})
        for element in fiche:
            feature = element.get_text().split(': ')
            feature_name = feature[0]
            if feature_name=="Centre(s) d'intérêt ":
                metier[feature_name] = feature[1].replace(', ', ' - ').split(',')
            elif feature_name=="Secteur(s) professionnel(s) ":
                feature_content = feature[1].split(',')
                if "Construction navale" in feature_content:
                    feature_content.append("Construction navale, ferroviaire et aéronautique")
                    feature_content.remove("Construction navale")
                    feature_content.remove(" ferroviaire et aéronautique")
                if "multisecteurs" in feature_content:
                    feature_content.remove("multisecteurs")
                metier[feature_name] = feature_content
                
            elif feature_name=="Statut(s) ":
                metier[feature_name] = feature[1].split(',')
            else:
                metier[feature_name] = feature[1].split(', ')
        
        # Scrap du "chapo" de la fiche métier
        chapo = soup.find(name = 'div', attrs={'class':'oni_chapo'})
        metier["chapo"] = chapo.get_text().replace('\xa0',' ')
        
        # Scrap de l'onglet 1 : "Le métier"
        onglet_1 = soup.find(name = 'div', attrs={'id':'oni_onglet-1','class':'oni_onglet'})
        content = []
        for element in onglet_1:
            content.append(element.get_text())
        metier["Le métier"] = content
        
        # Scrap de l'onglet 2 : "Où l'exercer"
        onglet_2 = soup.find(name = 'div', attrs={'id':'oni_onglet-2','class':'oni_onglet'})
        content = []
        for element in onglet_2:
            content.append(element.get_text())
        metier["Où l'exercer"] = content
        
        # Scrap de l'onglet 3 : "Carrière et salaire"
        onglet_3 = soup.find(name = 'div', attrs={'id':'oni_onglet-3','class':'oni_onglet'})
        content = []
        for element in onglet_3:
            content.append(element.get_text())
        metier["Carrière et salaire"] = content
        
        # Scrap de l'onglet 4 : "Accès au métier"
        onglet_4 = soup.find(name = 'div', attrs={'id':'oni_onglet-4','class':'oni_onglet'})
        content = []
        for element in onglet_4:
            content.append(element.get_text())
        metier["Accès au métier"] = content
        
        return(metier)
    except:
        print(link)
        return('error')
    
parse_job_page('http://www.onisep.fr/Ressources/Univers-Metier/Metiers/assistant-realisateur-assistante-realisatrice')

{'Accès au métier': ['Accès au métier',
  'Accès au métier',
  'Une solide formation technique et artistique est nécessaire pour devenir assistant réalisateur.',
  "Les écoles supérieures publiques qui recrutent sur concours constituent la voie classique. L'École nationale supérieure des métiers de l'image et du son (ENSMIS, plus connue sous le nom de FEMIS) et l'École nationale supérieure Louis Lumière (ENSLL) proposent des cursus de bac + 2 à bac + 5.masterà l'université.",
  '',
  'Niveau bac + 2',
  "BTS Métiers de l'audiovisuel ;",
  '',
  'Niveau bac + 3',
  "Licences sciences et technologies, mention technologie des arts, de l'image et du son ;Diplôme d'études supérieures de réalisation audiovisuelle, option cinéma (DESRA) ;Certificat de fin d'études de l'ESRA Bretagne option réalisation cinéma / vidéo ;Diplôme européen de réalisation audiovisuelle ;Formation d'assistant réalisateur (Conservatoire libre du cinéma français)Formation cinéma et audiovisuel spécialisation réalisatio

In [16]:
# Boucle pour scraper tous les métiers et les copier au format JSON dans le dossier 'Scraping output'
for row in metiers_links.itertuples():
    name = row[1]
    link = row[2]
    metier = parse_job_page(link)
    identifiant = link[-5:-1] + link[-1]
    if identifiant[0]!='1':
        identifiant = '1' + identifiant
    metier_output_path = os.path.join(output_dir_path, identifiant) + '.json'
    #print(metier_output_path)
    with open(metier_output_path, 'w') as f:
        json.dump(metier, f, indent=4)
    

TypeError: <h2 class="oni_hide">Carrière et salaire</h2> is not JSON serializable